In [1]:
import tensorflow as tf
import numpy as np

/Users/seungbaeji/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Recap

1. TensorFlow Program: Build a Computation Graph (tf.Graph)  
    1-1. Graph: Tensors + Operations
2. TensorFlow Runtime: Run the Graph using a Session (tf.Session)
3. Training: 
    - Graph: 
        - Reset Graph $\rightarrow$ Define a Pipeline $\rightarrow$ Define a Model $\rightarrow$ Define a Loss $\rightarrow$ Define a Optimizer $\rightarrow$ Op for init Variables
    - Session: 
        - Creating a Session $\rightarrow$ Feeding Data $\rightarrow$ Updating Variables

## Example for Recap

Let's recap the basic of TensorFlow through a model represnting the equation **$y = 2x + 1$**.

### TensorFlow Program

In [2]:
## Values for Model
data_x = np.arange(0, 1000, 5).reshape(-1, 1)
data_y = 2 * np.arange(0, 1000, 5).reshape(-1, 1) + 1
test_x = np.array([1,2,3,4,5,10,11,12,13,14,15]).reshape(-1, 1)
test_y = 2 * test_x + 1

## Normalization
data_x = data_x / 999
data_y = (data_y - 1) / 1999
test_x = test_x / 999

print('Shape of X:', data_x.shape, 
      '\nShape of Y:', data_y.shape, 
      '\nShape of Test:', test_x.shape)

# 1. Reset Graph
tf.reset_default_graph()

# 2. Define a Pipeline
## Placeholders
inputs = tf.placeholder(tf.float32, shape=(None, 1), name='input')
y_true = tf.placeholder(tf.float32, shape=(None, 1), name='label')

# 3. Define a Model
with tf.variable_scope('LinearModel') as vs:
    hidden = tf.layers.Dense(units=100)(inputs)
    y_pred = tf.layers.Dense(units=1)(hidden)

# 4. Define a Loss
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

# 5. Define a Optimizer
optimizer = tf.train.GradientDescentOptimizer(1e-3)
train = optimizer.minimize(loss)

# 6. Operation for Initialization of Variables
init = tf.global_variables_initializer()

# TensorBoard
writer = tf.summary.FileWriter('./recap')
writer.add_graph(tf.get_default_graph())
writer.flush()

Shape of X: (200, 1) 
Shape of Y: (200, 1) 
Shape of Test: (11, 1)


### TensorFlow Runtime

In [3]:
# 7. Define a Session
sess = tf.Session()

# 8. Init Variables
sess.run(init)

# 9. Feeding a Data
# 10. Updating Variables
for i in range(50000):
    _, loss_value = sess.run([train, loss], 
                             {inputs: data_x, y_true: data_y})
    if i % 10000 == 0:
        print(loss_value)

0.6374901
2.306999e-06
6.312609e-10
6.298135e-11
2.2465762e-11


In [9]:
# Test
test_result = sess.run(y_pred, feed_dict={inputs: test_x})

print('Model Result:', (test_result * 1999 + 1).flatten().astype(int))
print('Truth:', test_y.flatten())

Model Result: [ 3  5  7  9 11 21 23 25 27 29 31]
Truth: [ 3  5  7  9 11 21 23 25 27 29 31]


In [10]:
# 11. Release Resources
sess.close()

# Goal

TensorFlow에서 가장 많이 손보게 될 두 객체에 대해 다룬다.
- Tensor의 **모양을 확인**하는 방법, Tensor에서 **원하는 값을 선택**하는 방법, Tensor의 **데이터 타입을 변경**하는 방법.
- Variable에 **접근**하는 방법과 **값을 할당**하는 방법

# A. Tensor

- 값을 생성하기로 약속된 객체(원문 참조).
- 동일 데이터 타입을 담은 고차원 array로 표현. 
- 연산에 사용되는 값을 담은 객체이기 때문에, shape과 dtype이 주요 관심사.  
- 대부분의 operation들은 모양(shape)이 완전히 알려진 tensor를 출력.
- 하지만, 일부는 모양이 일부분만 알려진 tensor를 출력하며, 이때는 graph 실행시 모양을 알 수 있다.

> 원문: A `tf.Tensor` object represents a **partially defined computation that will eventually produce a value.**

## A.1) Rank
Tensor의 차원을 나타낸다.

> Note: Matrix의 rank와는 같지 않다.

|Rank | Math entity|
|-----|------------|
|0|	Scalar (magnitude only)|
|1|	Vector (magnitude and direction)|
|2|	Matrix (table of numbers)|
|3|	3-Tensor (cube of numbers)|
|n|	n-Tensor (you get the idea)|

### A.1.a) Rank 0

생성할 값만 입력하면 rank 0인 tensor가 생성.

In [2]:
# Reset graph
tf.reset_default_graph()

# Tensors
mammal = tf.constant("Elephant", tf.string)
ignition = tf.constant(451, tf.int16)
floating = tf.constant(3.14159265359, tf.float64)
its_complicated = tf.constant(12.3 - 4.85j, tf.complex64)

# Outputs of operations
print(mammal, ignition, floating, its_complicated, sep='\n')

# Tensor Values
with tf.Session() as sess:
    print(sess.run([mammal, ignition, floating, its_complicated]))

Tensor("Const:0", shape=(), dtype=string)
Tensor("Const_1:0", shape=(), dtype=int16)
Tensor("Const_2:0", shape=(), dtype=float64)
Tensor("Const_3:0", shape=(), dtype=complex64)
[b'Elephant', 451, 3.14159265359, (12.3-4.85j)]


### A.1.b) Rank 1

`list`로 값을 입력하면 rank 1인 tensor가 생성.

In [3]:
# Reset graph
tf.reset_default_graph()

# Tensors
mystr = tf.constant(["Hello"], tf.string)
cool_numbers  = tf.constant([3.14159, 2.71828], tf.float32)
first_primes = tf.constant([2, 3, 5, 7, 11], tf.int32)
its_very_complicated = tf.constant([12.3 - 4.85j, 7.5 - 6.23j], tf.complex64)

print(mystr, cool_numbers, first_primes, its_very_complicated, sep='\n')

# Run Graph
with tf.Session() as sess:
    print(sess.run([mystr, cool_numbers, first_primes, its_very_complicated]))

Tensor("Const:0", shape=(1,), dtype=string)
Tensor("Const_1:0", shape=(2,), dtype=float32)
Tensor("Const_2:0", shape=(5,), dtype=int32)
Tensor("Const_3:0", shape=(2,), dtype=complex64)
[array([b'Hello'], dtype=object), array([3.14159, 2.71828], dtype=float32), array([ 2,  3,  5,  7, 11], dtype=int32), array([12.3-4.85j,  7.5-6.23j], dtype=complex64)]


### A.1.c) Higher Ranks

row와 column이 존재하는 list를 입력하면 rank 2이상의 tensor가 생성.  
Rank는 그래프 실행시 `tf.rank`로 확인 가능하다.

In [4]:
tf.reset_default_graph()

# Tensors
mymat = tf.constant([[7],[11]], tf.int16)
myxor = tf.constant([[False, True],[True, False]], tf.bool)
linear_squares = tf.constant([[4], [9], [16], [25]], tf.int32)
squarish_squares = tf.constant([ [4, 9], [16, 25] ], tf.int32)
mymatC = tf.constant([[7],[11]], tf.int32)

rank_of_squares = tf.rank(squarish_squares)


print(mymat, myxor, linear_squares, squarish_squares, mymatC, sep='\n')
print(rank_of_squares)

with tf.Session() as sess:
    print(sess.run([mymat, myxor, linear_squares, squarish_squares, mymatC]))
    print('Rank of squares:', sess.run(rank_of_squares))

Tensor("Const:0", shape=(2, 1), dtype=int16)
Tensor("Const_1:0", shape=(2, 2), dtype=bool)
Tensor("Const_2:0", shape=(4, 1), dtype=int32)
Tensor("Const_3:0", shape=(2, 2), dtype=int32)
Tensor("Const_4:0", shape=(2, 1), dtype=int32)
Tensor("Rank:0", shape=(), dtype=int32)
[array([[ 7],
       [11]], dtype=int16), array([[False,  True],
       [ True, False]]), array([[ 4],
       [ 9],
       [16],
       [25]], dtype=int32), array([[ 4,  9],
       [16, 25]], dtype=int32), array([[ 7],
       [11]], dtype=int32)]
Rank of squares: 2


## A.2) Slice를 통한 tensor값 참조

slice를 통해 tensor의 일부분을 참조할 수 있다.

In [5]:
tf.reset_default_graph()

my_vector = tf.constant([1,2,3])
my_matrix = tf.constant([[1,2,3],[4,5,6],[7,8,9]])
print(my_vector, my_matrix, sep='\n')

# Slicing
my_scalar1 = my_vector[0]
my_scalar2 = my_matrix[1, 2]
print(my_scalar1, my_scalar2, sep='\n')

my_row_vector = my_matrix[2]
my_column_vector = my_matrix[:, 2]

with tf.Session() as sess:
    print('Scalar:', sess.run([my_scalar1, my_scalar2]))
    print('Vector:', sess.run([my_row_vector, my_column_vector]))

Tensor("Const:0", shape=(3,), dtype=int32)
Tensor("Const_1:0", shape=(3, 3), dtype=int32)
Tensor("strided_slice:0", shape=(), dtype=int32)
Tensor("strided_slice_1:0", shape=(), dtype=int32)
Scalar: [1, 6]
Vector: [array([7, 8, 9], dtype=int32), array([3, 6, 9], dtype=int32)]


## A.3) Shape
Tensor의 각 차원내 속한 값들의 갯수. Graph 생성시 자동으로 추측한다.  
These inferred shapes might have known or unknown rank. If the rank is known, the sizes of each dimension might be known or unknown.

The TensorFlow documentation uses three notational conventions to describe tensor dimensionality: rank, shape, and dimension number. The following table shows how these relate to one another:


|Rank | Shape | Dimension number | Example |
| ----------------------|
|0|[]|0-D|A 0-D tensor. A scalar.|
|1|[D0]|1-D|A 1-D tensor with shape [5].|
|2|[D0, D1]|2-D|A 2-D tensor with shape [3, 4].|
|3|[D0, D1, D2]|3-D|A 3-D tensor with shape [1, 4, 3].|
|n|[D0, D1, ... Dn-1]|n-D|A tensor with shape [D0, D1, ... Dn-1].|

### A.3.a) Tensor의 shape 획득

Tensor의 shape 획득에는 2가지 방법이 존재한다.
1. Graph 생성시 shape 획득
2. Graph 실행시 shape 획득

#### A.3.a-1) Graph 생성시 shape 획득

이미 모양을 알고 있는 tensor의 shape획득에 유용하다. `tf.Tensor`객체의 shape 속성을 읽어, `TensorShape`객체로 반환한다.   

#### A.3.a-2) Graph 실행시 shape 획득

특정 `tf.Tensor`의 shape을 획득할 수 있는 `tf.shape` operation으로 획득. 
해당 operation은 `tf.Tensor`을 출력한다.

In [6]:
tf.reset_default_graph()

my_vector = tf.constant([1,2,3])
my_matrix = tf.constant([[1,2,3],[4,5,6],[7,8,9]])

print('Case 1:')
print('Type:', type(my_vector.shape), '/ Shape:', my_vector.shape)
print('Type:', type(my_matrix.shape), '/ Shape:', my_matrix.shape)

shape_vec = tf.shape(my_vector)
shape_mat = tf.shape(my_matrix)

writer = tf.summary.FileWriter('./shape')
writer.add_graph(tf.get_default_graph())
writer.flush()

with tf.Session() as sess:
    print('\nCase 2:')
    print('Type:', type(shape_vec), '/ Shape:', sess.run(shape_vec))
    print('Type:', type(shape_mat), '/ Shape:', sess.run(shape_mat))

Case 1:
Type: <class 'tensorflow.python.framework.tensor_shape.TensorShape'> / Shape: (3,)
Type: <class 'tensorflow.python.framework.tensor_shape.TensorShape'> / Shape: (3, 3)

Case 2:
Type: <class 'tensorflow.python.framework.ops.Tensor'> / Shape: [3]
Type: <class 'tensorflow.python.framework.ops.Tensor'> / Shape: [3 3]


### A.3.b) Tensor의 shape 변경

`tf.reshape`을 통해서 변경이 가능하다.

In [7]:
tf.reset_default_graph()

rank_three_tensor = tf.ones([3, 4, 5])
print(rank_three_tensor)

matrix = tf.reshape(rank_three_tensor, [6, 10])  # Reshape existing content into
                                                 # a 6x10 matrix
print(matrix)

matrixB = tf.reshape(matrix, [3, -1])  #  Reshape existing content into a 3x20
                                       # matrix. -1 tells reshape to calculate
                                       # the size of this dimension.
print(matrixB)
        
matrixAlt = tf.reshape(matrixB, [4, 3, -1])  # Reshape existing content into a
                                             #4x3x5 tensor
print(matrixAlt)

# Note that the number of elements of the reshaped Tensors has to match the
# original number of elements. Therefore, the following example generates an
# error because no possible value for the last dimension will match the number
# of elements.
yet_another = tf.reshape(matrixAlt, [13, 2, -1])  # ERROR!

Tensor("ones:0", shape=(3, 4, 5), dtype=float32)
Tensor("Reshape:0", shape=(6, 10), dtype=float32)
Tensor("Reshape_1:0", shape=(3, 20), dtype=float32)
Tensor("Reshape_2:0", shape=(4, 3, 5), dtype=float32)


ValueError: Dimension size must be evenly divisible by 26 but is 60 for 'Reshape_3' (op: 'Reshape') with input shapes: [4,3,5], [3] and with input tensors computed as partial shapes: input[1] = [13,2,?].

## A.4) Tensor의 데이터타입 변경

`tf.cast`를 이용하여, 데이터타입이 변경된 tensor의 획득이 가능하다.

#### 데이터 타입
1. `tf.float16`: 16-bit half-precision floating-point.
2. `tf.float32`: 32-bit single-precision floating-point.
3. `tf.float64`: 64-bit double-precision floating-point.
4. `tf.bfloat16`: 16-bit truncated floating-point.
3. `tf.complex64`: 64-bit single-precision complex.
3. `tf.complex128`: 128-bit double-precision complex.
3. `tf.int8`: 8-bit signed integer.
3. `tf.uint8`: 8-bit unsigned integer.
3. `tf.uint16`: 16-bit unsigned integer.
3. `tf.uint32`: 32-bit unsigned integer.
3. `tf.uint64`: 64-bit unsigned integer.
3. `tf.int16`: 16-bit signed integer.
3. `tf.int32`: 32-bit signed integer.
3. `tf.int64`: 64-bit signed integer.
3. `tf.bool`: Boolean.
3. `tf.string`: String.
3. `tf.qint8`: Quantized 8-bit signed integer.
3. `tf.quint8`: Quantized 8-bit unsigned integer.
3. `tf.qint16`: Quantized 16-bit signed integer.
3. `tf.quint16`: Quantized 16-bit unsigned integer.
3. `tf.qint32`: Quantized 32-bit signed integer.
3. `tf.resource`: Handle to a mutable resource.
3. `tf.variant`: Values of arbitrary types.

In [8]:
tf.reset_default_graph()

# Int tensor
int_tensor = tf.constant([0, 1, 2, 3], dtype=tf.int32)
print(int_tensor)

# Casting
float_tensor = tf.cast(int_tensor, dtype=tf.float32)
bool_tensor = tf.cast(int_tensor, dtype=tf.bool)
print(int_tensor, float_tensor, bool_tensor)

with tf.Session() as sess:
    print(sess.run([float_tensor, bool_tensor]))

Tensor("Const:0", shape=(4,), dtype=int32)
Tensor("Const:0", shape=(4,), dtype=int32) Tensor("Cast:0", shape=(4,), dtype=float32) Tensor("Cast_1:0", shape=(4,), dtype=bool)
[array([0., 1., 2., 3.], dtype=float32), array([False,  True,  True,  True])]


### A.5)  `tf.Tensor` 객체의 속성
1. **dtype**: 해당 Tensor내 요소들의 데이터 타입.
2. **shape**: 해당 Tensor의 모양.
3. device: 해당 Tensor가 생성되기로 한 장비의 이름.
4. graph: 해당 Tensor가 속한 그래프.
5. name: 해당 Tensor의 이름.
6. op: 해당 Tensor를 출력하는 operation.

# B. Variables

TensorFlow에서 variable은 Program에 의해 관리되는 공유되고 보존되는 상태를 표현하는데 가장 좋은 방법이다.(원문참조)
Variable들은 `tf.Variable` 클래스를 통해 다뤄진다. `tf.Variable`는 operation들을 해당 객체에 실행시켜 값을 변환할 수 있는 tensor를 나타낸다. `tf.Tensor` 객체와 달리, 하나의 `sessesion.run` call 밖에서 관리된다.

Internally, a tf.Variable stores a persistent tensor. Specific ops allow you to read and modify the values of this tensor. These modifications are visible across multiple tf.Sessions, so multiple workers can see the same values for a tf.Variable.

> 원문: A TensorFlow variable is the best way to represent shared, persistent state manipulated by your program.


## B.1) Variable 생성

variable을 만드는 가장좋은 방법은 `tf.get_variable`함수를 호출하는 것이다.  
기본적으로, 해당함수는 정해진 모양(shape)의 값들을 `tf.float32`의 데이터 타입으로 랜덤하게 초기화 한다.  
그리고 기본적인 값의 초기화 함수는 `tf.glorot_uniform_initializer`을 이용한다.

기본값 이외에 옵션을 주어 설정이 가능하다.

> Note: initializer로 `tf.Tensor`를 이용시, shape을 지정하면 안된다. Variable의 shape으론, 해당 tensor의 shape이 사용된다.

The best way to create a variable is to call the tf.get_variable function. This function requires you to specify the Variable's name. This name will be used by other replicas to access the same variable, as well as to name this variable's value when checkpointing and exporting models. tf.get_variable also allows you to reuse a previously created variable of the same name, making it easy to define models which reuse layers.

In [9]:
tf.reset_default_graph()

# Random values
my_variable = tf.get_variable("my_variable", [1, 2, 3])
print(my_variable)

# Zero values
my_int_variable = tf.get_variable("my_int_variable", [1, 2, 3], dtype=tf.int32,
  initializer=tf.zeros_initializer)
print(my_int_variable)

# Constants
other_variable = tf.get_variable("other_variable", dtype=tf.int32,
  initializer=tf.constant([23, 42]))
print(other_variable)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('my_variable: ', sess.run(my_variable))
    print('my_variable: ', sess.run(my_variable))
    print('my_int_variable: ', sess.run(my_int_variable))
    print('other_variable: ', sess.run(other_variable))

<tf.Variable 'my_variable:0' shape=(1, 2, 3) dtype=float32_ref>
<tf.Variable 'my_int_variable:0' shape=(1, 2, 3) dtype=int32_ref>
<tf.Variable 'other_variable:0' shape=(2,) dtype=int32_ref>
my_variable:  [[[-1.0450147   0.7702333  -0.83383346]
  [-0.04738069 -0.7891766   0.8842331 ]]]
my_variable:  [[[-1.0450147   0.7702333  -0.83383346]
  [-0.04738069 -0.7891766   0.8842331 ]]]
my_int_variable:  [[[0 0 0]
  [0 0 0]]]
other_variable:  [23 42]


## B.2) Variable Collections

기본적으로 모든 `tf.Variable`들은 아래의 두 collections에 존재한다.

- `tf.GraphKeys.GLOBAL_VARIABLES`: 여러 기기들간에 공유될 수 있는 `variable`들
- `tf.GraphKeys.TRAINABLE_VARIABLES`: TensorFlow가 gradient를 계산할 수 있는 `variable`들

`variable`들이 학습(업데이트)되는것을 원치 않는다면, `tf.GraphKeys.LOCAL_VARIABLES`에 추가하여야 한다.  
혹은 `variable`생성시 `trainable=False`로 하여야 한다.

In [11]:
tf.reset_default_graph()

my_local = tf.get_variable("my_local", shape=(),
collections=[tf.GraphKeys.LOCAL_VARIABLES])

my_non_trainable = tf.get_variable("my_non_trainable",
                                   shape=(),
                                   trainable=False)

my_trainable = tf.get_variable("my_trainable",
                                shape=(),
                                trainable=True)

tf.add_to_collection("my_collection_name", my_local)

print('Custom collection: ', tf.get_collection("my_collection_name"))
print('Local Variables: ', tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES))
print('Global Variables: ', tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))
print('Trainable Variables: ', tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

Custom collection:  [<tf.Variable 'my_local:0' shape=() dtype=float32_ref>]
Local Variables:  [<tf.Variable 'my_local:0' shape=() dtype=float32_ref>]
Global Variables:  [<tf.Variable 'my_non_trainable:0' shape=() dtype=float32_ref>, <tf.Variable 'my_trainable:0' shape=() dtype=float32_ref>]
Trainable Variables:  [<tf.Variable 'my_local:0' shape=() dtype=float32_ref>, <tf.Variable 'my_trainable:0' shape=() dtype=float32_ref>]


## B.3) Variable 초기화

`variable`을 사용전에 **반드시 초기화** 시켜주어야 한다.
한번에 **학습가능한 `variable`들**을 초기화 시키려면, `tf.global_variables_initializer()`를 사용하면 된다. 해당 함수는 `tf.GraphKeys.GLOBAL_VARIABLES` collection에 있는 `variable`들을 초기화 시켜주는 operation을 리턴한다.

In [12]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())
print(sess.run([my_non_trainable, my_trainable]))
print(sess.run(my_local))  # ERROR!!

[-0.78687054, 0.2129265]


FailedPreconditionError: Attempting to use uninitialized value my_local
	 [[{{node _retval_my_local_0_0}} = _Retval[T=DT_FLOAT, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](my_local)]]

### B.3-a) 초기화 되지 않은 variable 확인하기

`tf.report_uninitialized_variables()`함수는 초기화 되지 않은 `variable`들을 알려준다.

In [13]:
sess.run(tf.report_uninitialized_variables())

array([b'my_local'], dtype=object)

In [14]:
sess.run(my_local.initializer)
sess.run(my_local)

-0.43277323

In [15]:
sess.run(tf.report_uninitialized_variables())

array([], dtype=object)

### B.3-b) 초기화의 주의점

`tf.global_variables_initializer()`는 variable들의 순서를 지정하여 초기화 하지 않는다. 때문에 초기화 값이 다른 variable의 값과 연관되어 있다면, 에러가 발생할 확률이 높다. 초기화 값은 `my_varible.initialized_value()`을 이용한다.

> 참고. 간단한 테스트에서는 에러발생을 보이지 않았다.

In [16]:
tf.reset_default_graph()

# Random values
var_1 = tf.get_variable("var_1", shape=(), initializer=tf.zeros_initializer())
var_2 = tf.get_variable("var_2", dtype=tf.float32, initializer=var_1 + 1.0)  # ERROR???
print(var_1, var_2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('Var_1: ', sess.run(var_1))
    print('Var_2: ', sess.run(var_2))


tf.reset_default_graph()

var_1 = tf.get_variable("var_1", shape=(), initializer=tf.zeros_initializer())
var_2 = tf.get_variable("var_2", initializer=var_1.initialized_value() + 1)
print(var_1, var_2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('Var_1: ', sess.run(var_1))
    print('Var_2: ', sess.run(var_2))

<tf.Variable 'var_1:0' shape=() dtype=float32_ref> <tf.Variable 'var_2:0' shape=() dtype=float32_ref>
Var_1:  0.0
Var_2:  1.0
<tf.Variable 'var_1:0' shape=() dtype=float32_ref> <tf.Variable 'var_2:0' shape=() dtype=float32_ref>
Var_1:  0.0
Var_2:  1.0


## B.4) Variable 사용

TensorFlow 그래프에서 `tf.Variable`의 값을 사용하기 위해서는 `tf.Tensor`처럼 다루면 된다.  
Variable에 값을 할당하려면 `tf.Variable` 인스턴스에서 assign, assign_add 같은 메소드를 사용하면된다.

In [17]:
tf.reset_default_graph()

v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
w = v + 1

print(v)
print(w)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run([v, w]))

<tf.Variable 'v:0' shape=() dtype=float32_ref>
Tensor("add:0", shape=(), dtype=float32)
[0.0, 1.0]


In [18]:
tf.reset_default_graph()

v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
assignment = v.assign_add(1)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(v))
    print(sess.run([v, assignment]))
    print(sess.run(v))

0.0
[1.0, 1.0]
1.0


In [190]:
tf.reset_default_graph()

v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
assignment = v.assign_add(1)
with tf.control_dependencies([assignment]):
    w = v.read_value()
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(v))
    print(sess.run([v, w]))
    print(sess.run(w))

0.0
[1.0, 1.0]
2.0


## B.5) Sharing variables

Variable을 공유하는 방법은 2가지가 존재한다.
1. `tf.Variable` 객체를 직접 입력(Explicit)
2. `tf.Varibale` 객체를 `tf.variable_scope` 객체 안에 wrapping(Implicit)



In [44]:
def conv_relu(input, kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape,
        initializer=tf.random_normal_initializer())
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape,
        initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights,
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

아래 코드에서, 마지막 줄은 "weights", "biases" `variable`들이 이미 존재하기 때문에 실패한다.

In [45]:
tf.reset_default_graph()

input1 = tf.random_normal([1,10,10,32])
input2 = tf.random_normal([1,20,20,32])
x = conv_relu(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])
y = conv_relu(x, kernel_shape=[5, 5, 32, 32], bias_shape = [32])  # This fails.

ValueError: Variable weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-44-5edf5d2e6e72>", line 4, in conv_relu
    initializer=tf.random_normal_initializer())
  File "<ipython-input-45-3b9dfce2e618>", line 5, in <module>
    x = conv_relu(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])
  File "/Users/seungbaeji/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [46]:
print('Global Variables: ', tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

Global Variables:  [<tf.Variable 'weights:0' shape=(5, 5, 32, 32) dtype=float32_ref>, <tf.Variable 'biases:0' shape=(32,) dtype=float32_ref>]


In [47]:
def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])

In [48]:
with tf.variable_scope("model"):
    output1 = my_image_filter(input1)
with tf.variable_scope("model", reuse=True):
    output2 = my_image_filter(input2)

In [49]:
print('Global Variables: ', tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

Global Variables:  [<tf.Variable 'weights:0' shape=(5, 5, 32, 32) dtype=float32_ref>, <tf.Variable 'biases:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'model/conv1/weights:0' shape=(5, 5, 32, 32) dtype=float32_ref>, <tf.Variable 'model/conv1/biases:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'model/conv2/weights:0' shape=(5, 5, 32, 32) dtype=float32_ref>, <tf.Variable 'model/conv2/biases:0' shape=(32,) dtype=float32_ref>]


In [50]:
tf.reset_default_graph()

with tf.variable_scope("model") as scope:
    output1 = my_image_filter(input1)
    scope.reuse_variables()
    output2 = my_image_filter(input2)

ValueError: Tensor("model/conv1/weights:0", shape=(5, 5, 32, 32), dtype=float32_ref) must be from the same graph as Tensor("random_normal:0", shape=(1, 10, 10, 32), dtype=float32).

In [28]:
with tf.variable_scope("model") as scope:
    output1 = my_image_filter(input1)
with tf.variable_scope(scope, reuse=True):
    output2 = my_image_filter(input2)


ValueError: Variable model/conv1/weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-19-5edf5d2e6e72>", line 4, in conv_relu
    initializer=tf.random_normal_initializer())
  File "<ipython-input-24-3e2611218a85>", line 4, in my_image_filter
    relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
  File "<ipython-input-25-af479b636c15>", line 2, in <module>
    output1 = my_image_filter(input1)


# Summary

`Tensor`와 `Variable`은 TebsorFlow 프로그램 작성시 가장 흔하게 다루게 되는 객체이다.  
- `Tensor`는 모델에 입력(`tf.data`)되거나 모델에 흐르는 데이터를 다룰때 자주 접근하게 되고,
- `Variable`은 모델의 parameter(ex. LSTM의 state, global steps)에 접근하여 이를 수정하거나 관리할 때 다루게 된다.

`Tensor`는 shape과 dtype에 신경을 자주 쓰게 되며, `Variable`은 collection별 초기화 및 값 할당을 하게 된다.

# References

1. [TensorFlow Guide: Tensors](https://www.tensorflow.org/guide/tensors)
2. [TensorFlow API: tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)
3. [Persistent Data Structure](https://en.wikipedia.org/wiki/Persistent_data_structure)

## Executing computations

1. 